In [674]:
import pandas as pd

accidents_df = pd.read_csv('dataset/US_Accidents_Dec21_updated.csv')
accidents_copy = accidents_df.copy()

Visualizzazione delle caratteristiche del Dataset: numero di righe, numero di features e output a video delle prime 10 righe del dataset

In [675]:
accidents_copy.shape

(2845342, 47)

In [676]:
accidents_copy.drop('ID',axis=1,inplace=True)

In [677]:
from imblearn.over_sampling._smote.base import Counter

lista_da_eliminare = []
lista_check = []
lista_weather = Counter(accidents_copy['Weather_Condition'])

for elemento in lista_weather.keys():
    if lista_weather[elemento] < 100:
        lista_check.append(elemento)

for index,row in accidents_copy.iterrows():
    indice = row['Severity']
    stringa = row['Start_Time'].split('-')
    stringa = stringa[0]
    if stringa == '2021' or stringa == '2020' or indice == 1 or indice == 4:
        lista_da_eliminare.append(index)
        

In [678]:
accidents_copy.drop(labels=lista_da_eliminare,inplace=True)

In [679]:
accidents_copy.drop('Country',axis=1,inplace=True)
accidents_copy.drop('State',axis=1,inplace=True)
accidents_copy.drop('Wind_Direction',axis=1,inplace=True)
accidents_copy.drop('Nautical_Twilight',axis=1,inplace=True)

In [680]:
accidents_copy.drop('City',axis=1,inplace=True)
accidents_copy.drop('County',axis=1,inplace=True)
accidents_copy.drop('End_Time',axis=1,inplace=True)
accidents_copy.drop('Street',axis=1,inplace=True)
accidents_copy.drop('Description',axis=1,inplace=True)
accidents_copy.drop('Roundabout',axis=1,inplace=True)
accidents_copy.drop('Traffic_Calming',axis=1,inplace=True)
accidents_copy.drop('Turning_Loop',axis=1,inplace=True)
accidents_copy.drop('Astronomical_Twilight',axis=1,inplace=True)
accidents_copy.drop('Bump',axis=1,inplace=True)
accidents_copy.drop('Civil_Twilight',axis=1,inplace=True)
accidents_copy.drop('Give_Way',axis=1,inplace=True)
accidents_copy.drop('No_Exit',axis=1,inplace=True)

In [681]:
accidents_copy.drop('Timezone',axis=1,inplace=True)
accidents_copy.drop('Airport_Code',axis=1,inplace=True)

Controllo di valori null, valori mancanti e altre inconsistency nel dataset

In [682]:
accidents_copy.drop('Zipcode',axis = 1,inplace=True)
check = accidents_copy.isna().sum()

In [683]:
accidents_copy.isna().sum()

Severity                  0
Start_Time                0
Start_Lat                 0
Start_Lng                 0
End_Lat                   0
End_Lng                   0
Distance(mi)              0
Number               505269
Side                      0
Weather_Timestamp      9231
Temperature(F)        15420
Wind_Chill(F)        351617
Humidity(%)           16059
Pressure(in)          13027
Visibility(mi)        16547
Wind_Speed(mph)       82551
Precipitation(in)    385856
Weather_Condition     16736
Amenity                   0
Crossing                  0
Junction                  0
Railway                   0
Station                   0
Stop                      0
Traffic_Signal            0
Sunrise_Sunset           24
dtype: int64

In [684]:
columns_names = accidents_copy.columns.values.tolist()
na_columns = []

for index in range(len(check)):
    if check[index] != 0:
        na_columns.append(columns_names[index])

Qui otteniamo i nomi delle colonne che contengono null values da gestire in modo da processare le singole colonne indipendentemente

In [685]:
print(na_columns)
len_na_columns = len(na_columns)
index_na_columns = 0
print(len_na_columns)

['Number', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Sunrise_Sunset']
11


In [686]:
def examin_column(current_index):
    column_name = na_columns[current_index]
    accidents_copy[column_name].head(20)
    column_na_count = accidents_copy[column_name].isna().sum()
    ratio = column_na_count/accidents_copy.shape[0]
    print("Total count of null values for the column " + str(column_name) + " = " + str(column_na_count))
    print("Ratio between total number of values of the column respect to its null values = " + str(ratio))


Some considerations on the column: Number is a numerical attribute and has we can see it has more than hal of the value null. We could consider to replace the null values but the best alternative is to drop the column because it is useless in our problem: it just shows the street number

In [687]:
accidents_final_df = accidents_copy
accidents_final_df.drop('Number',axis = 1,inplace=True)
accidents_final_df.drop('Weather_Timestamp',axis = 1,inplace=True)
index_na_columns += 2

In [688]:
accidents_final_df.shape

(622749, 24)

In [689]:
examin_column(index_na_columns)

Total count of null values for the column Temperature(F) = 15420
Ratio between total number of values of the column respect to its null values = 0.024761179865403236


In [690]:
def examin_column_final(current_index):
    if current_index >= len_na_columns:
        return 
    column_name = na_columns[current_index]
    accidents_final_df[column_name].head(20)
    column_na_count = accidents_final_df[column_name].isna().sum()
    ratio = column_na_count/accidents_final_df.shape[0]
    print("Total count of null values for the column " + str(column_name) + " = " + str(column_na_count))
    print("Ratio between total number of values of the column respect to its null values = " + str(ratio))


In [691]:
temp = accidents_final_df[accidents_final_df[na_columns[index_na_columns]].isnull()]
index_array = temp.index
accidents_final_df.drop(index_array,axis = 0,inplace=True)
examin_column_final(index_na_columns)
index_na_columns += 1

Total count of null values for the column Temperature(F) = 0
Ratio between total number of values of the column respect to its null values = 0.0


In [692]:
examin_column_final(index_na_columns)
index_na_columns += 1

Total count of null values for the column Wind_Chill(F) = 336197
Ratio between total number of values of the column respect to its null values = 0.5535665183121504


In [693]:
accidents_final_df.rename(columns = {'Wind_Chill(F)':'Wind_Chill'}, inplace = True)
print(accidents_final_df.Wind_Chill.mean())
accidents_final_df['Wind_Chill'].fillna(accidents_final_df.Wind_Chill.mean(),inplace= True)

51.88712213976956


Anche in questo caso si parla di un numero infinitesimale di tuple, l'alternativa migliore è eliminarle, la percentuale di perdita è minima

In [694]:
examin_column_final(index_na_columns)

Total count of null values for the column Humidity(%) = 639
Ratio between total number of values of the column respect to its null values = 0.0010521480120330167


Stesso discorso, in questo caso può essere droppata completamente la feature in quanto non rilevante per il nostro problema (lo zipcode è il codice di avviamento postale) 

Droppo le righe direttamente

In [695]:
temp = accidents_final_df[accidents_final_df[na_columns[index_na_columns]].isnull()]
index_array = temp.index
accidents_final_df.drop(index_array,axis = 0,inplace=True)
examin_column_final(index_na_columns)
index_na_columns += 1

Total count of null values for the column Humidity(%) = 0
Ratio between total number of values of the column respect to its null values = 0.0


In [696]:
examin_column_final(index_na_columns)
index_na_columns += 1

Total count of null values for the column Pressure(in) = 923
Ratio between total number of values of the column respect to its null values = 0.0015213700571956024


In [697]:
accidents_final_df.shape

(606690, 24)

Droppo le righe

In [698]:
accidents_final_df.rename(columns = {'Pressure(in)':'Pressure'}, inplace = True)
print(accidents_final_df.Pressure.mean())
accidents_final_df['Pressure'].fillna(accidents_final_df.Pressure.mean(),inplace= True)

29.726556184143394


In [699]:
examin_column_final(index_na_columns)
index_na_columns += 1

Total count of null values for the column Visibility(mi) = 3719
Ratio between total number of values of the column respect to its null values = 0.006129984011603949


Droppo la colonna

In [700]:
accidents_final_df.rename(columns = {'Visibility(mi)':'Visibility'}, inplace = True)
print(accidents_final_df.Visibility.mean())
accidents_final_df['Visibility'].fillna(accidents_final_df.Visibility.mean(),inplace= True)

9.202145327055531


In [701]:
examin_column_final(index_na_columns)
index_na_columns += 1

Total count of null values for the column Wind_Speed(mph) = 69177
Ratio between total number of values of the column respect to its null values = 0.11402363645354299


In [702]:
accidents_final_df.rename(columns = {'Temperature(F)':'Temperature'}, inplace = True)
print(accidents_final_df.Temperature.mean())
accidents_final_df['Temperature'].fillna(accidents_final_df.Temperature.mean(),inplace= True)

61.554962501442255


In [703]:
column_na_count = accidents_final_df['Temperature'].isna().sum()
ratio = column_na_count/accidents_final_df.shape[0]
print("Total count of null values for the column " + str('Temperature') + " = " + str(column_na_count))
print("Ratio between total number of values of the column respect to its null values = " + str(ratio))

Total count of null values for the column Temperature = 0
Ratio between total number of values of the column respect to its null values = 0.0


Temperature è stata rinominata e i valori nulla sono stati gestiti con la media della colonna

In [704]:
index_na_columns += 1
examin_column_final(index_na_columns)

Total count of null values for the column Weather_Condition = 4019
Ratio between total number of values of the column respect to its null values = 0.006624470487398836


In [705]:
accidents_final_df.rename(columns = {'Wind_Speed(mph)':'Wind_Speed'}, inplace = True)
print(accidents_final_df.Wind_Speed.mean())
accidents_final_df['Wind_Speed'].fillna(accidents_final_df.Wind_Speed.mean(),inplace= True)

8.314610437328957


In [706]:
column_na_count = accidents_final_df['Wind_Speed'].isna().sum()
ratio = column_na_count/accidents_final_df.shape[0]
print("Total count of null values for the column " + str('Wind_Speed') + " = " + str(column_na_count))
print("Ratio between total number of values of the column respect to its null values = " + str(ratio))

Total count of null values for the column Wind_Speed = 0
Ratio between total number of values of the column respect to its null values = 0.0


Anche Wind_Chill(F) è stato rinominato e ai valori nulli è stata applicata la media della feature

In [707]:
accidents_final_df.rename(columns = {'Precipitation(in)':'Precipitation'}, inplace = True)
print(accidents_final_df.Precipitation.mean())
accidents_final_df['Precipitation'].fillna(accidents_final_df.Precipitation.mean(),inplace= True)

0.018501980493206695


In [708]:
accidents_final_df.rename(columns = {'Humidity(%)':'Humidity'}, inplace = True)
column_na_count = accidents_final_df['Humidity'].isna().sum()
ratio = column_na_count/accidents_final_df.shape[0]
print("Total count of null values for the column " + str('Humidity') + " = " + str(column_na_count))
print("Ratio between total number of values of the column respect to its null values = " + str(ratio))

Total count of null values for the column Humidity = 0
Ratio between total number of values of the column respect to its null values = 0.0


Anche Humidity(%) è stato rinominato e ai valori nulli è stata applicata la media della feature

In [709]:
column_na_count = accidents_final_df['Visibility'].isna().sum()
ratio = column_na_count/accidents_final_df.shape[0]
print("Total count of null values for the column " + str('Visibility') + " = " + str(column_na_count))
print("Ratio between total number of values of the column respect to its null values = " + str(ratio))

Total count of null values for the column Visibility = 0
Ratio between total number of values of the column respect to its null values = 0.0


Anche Visibility(mi) è stata rinominata e messa la media della feature ai valori nulli

In [710]:
def visualize_renamed_column(df,column):
    column_na_count = df[column].isna().sum()
    ratio = column_na_count/df.shape[0]
    print("Total count of null values for the column " + str(column) + " = " + str(column_na_count))
    print("Ratio between total number of values of the column respect to its null values = " + str(ratio))

In [711]:
visualize_renamed_column(accidents_final_df,'Wind_Speed')

Total count of null values for the column Wind_Speed = 0
Ratio between total number of values of the column respect to its null values = 0.0


In [712]:
visualize_renamed_column(accidents_final_df,'Precipitation')

Total count of null values for the column Precipitation = 0
Ratio between total number of values of the column respect to its null values = 0.0


In [713]:
visualize_renamed_column(accidents_final_df,'Humidity')

Total count of null values for the column Humidity = 0
Ratio between total number of values of the column respect to its null values = 0.0


Anche di Precipitation ho fatto la media

In [714]:
accidents_final_df.Weather_Condition.mode()[0]
accidents_final_df.Weather_Condition.fillna(accidents_final_df.Weather_Condition.mode()[0], inplace=True)

In [715]:
temp = accidents_final_df[accidents_final_df[na_columns[index_na_columns]].isnull()]
index_array = temp.index
accidents_final_df.drop(index_array,axis = 0,inplace=True)

In [716]:
temp = accidents_final_df[accidents_final_df['Sunrise_Sunset'].isnull()]
index_array = temp.index
accidents_final_df.drop(index_array,axis = 0,inplace=True)

Weather_Condition: null values sostituiti con il valore più frequente

Per Sunrise_Sunset ho tolto le righe con i valori null essendo meno di 3000. Inoltre così facendo si può notare che si sono tolti i valori null anche delle rimanenti colonne da esaminare. Quindi avevano i valori null tutte alle stesse righe. I valori mancanti quindi sono stati gestiti

In [717]:
accidents_final_df.isna().sum().sum()

0

In [718]:
accidents_final_df.isna().sum()

Severity             0
Start_Time           0
Start_Lat            0
Start_Lng            0
End_Lat              0
End_Lng              0
Distance(mi)         0
Side                 0
Temperature          0
Wind_Chill           0
Humidity             0
Pressure             0
Visibility           0
Wind_Speed           0
Precipitation        0
Weather_Condition    0
Amenity              0
Crossing             0
Junction             0
Railway              0
Station              0
Stop                 0
Traffic_Signal       0
Sunrise_Sunset       0
dtype: int64

DIMENSIONALITY REDUCTION
Alcune feature non sono utili alla causa quindi le droppo

In [719]:
accidents_without_null = accidents_final_df.copy() 

In [720]:
from datetime import datetime
colonna_giorni = accidents_without_null['Start_Time']
colonna_giorni_df = pd.DataFrame(accidents_without_null['Start_Time'])

In [721]:

for index,row in colonna_giorni_df.iterrows():
    stringa = row[0]
    stringa = stringa.split('.')
    datetime_object = datetime.strptime(stringa[0], '%Y-%m-%d %H:%M:%S')
    colonna_giorni_df.loc[index][0] = datetime_object.weekday()
    

In [722]:
accidents_without_null['Start_Time'] = colonna_giorni_df

In [723]:
accidents_without_null.tail(5)

,Severity,Start_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Side,Temperature,Wind_Chill,...,Precipitation,Weather_Condition,Amenity,Crossing,Junction,Railway,Station,Stop,Traffic_Signal,Sunrise_Sunset
2845337,2,4,34.00248,-117.37936,33.99888,-117.37094,0.543,R,86.0,86.0,...,0.0,Fair,False,False,False,False,False,False,False,Day
2845338,2,4,32.76696,-117.14806,32.76555,-117.15363,0.338,R,70.0,70.0,...,0.0,Fair,False,False,False,False,False,False,False,Day
2845339,2,4,33.77545,-117.84779,33.77740,-117.85727,0.561,R,73.0,73.0,...,0.0,Partly Cloudy,False,False,True,False,False,False,False,Day
2845340,2,4,33.99246,-118.40302,33.98311,-118.39565,0.772,R,71.0,71.0,...,0.0,Fair,False,False,False,False,False,False,False,Day
2845341,2,4,34.13393,-117.23092,34.13736,-117.23934,0.537,R,79.0,79.0,...,0.0,Fair,False,False,False,False,False,False,False,Day


In [724]:
def examine_column_distribution(column):
    print("the feature " + str(column) + " is distributed this way: ")
    print(accidents_without_null[column].value_counts())


Regarding the features about twilights, they are different ways of considering the start of the day by different point of views:

Civil Twilight: morning civil twilight begins when the geometric center of the sun is 6 degrees below the horizon, and ends at sunrise
Nautical Twilight: Begins in the morning, or ends in the evening, when the geometric center of the sun is 12 degrees below the horizon
Astronomical Twilight: Begins in the morning, or ends in the evening, when the geometric center of the sun is 18 degrees below the horizon

The measure that is more fit to us is actually the Sunrise Sunset, which takes into account the start of the day by the rising of the sun. Therefore the Sunrise Sunset is kept while the others are dropped.

In [725]:
accidents_without_null.shape

(606668, 24)

In [726]:
def convert_boolean_column_to_int(column):
    temp = list(map(int,accidents_without_null[column]))
    accidents_without_null[column] = temp

In [727]:
columns_to_convert = ['Amenity','Crossing','Junction','Railway','Station','Stop','Traffic_Signal']

for column_target in columns_to_convert:
    convert_boolean_column_to_int(column_target)

In [728]:
def day_night(record):
    if record == 'Night':
        return 0
    elif record == 'Day':
        return 1

temp = list(map(day_night,accidents_without_null['Sunrise_Sunset']))
accidents_without_null['Sunrise_Sunset'] = temp

In [729]:
def day_night(record):
    if record == 'L':
        return 0
    elif record == 'R':
        return 1

temp = list(map(day_night,accidents_without_null['Side']))
accidents_without_null['Side'] = temp

In [730]:
accidents_without_null.shape

(606668, 24)

In [731]:
def change(record):
    if record == 2:
        return 0
    elif record == 3:
        return 1

temp = list(map(change,accidents_without_null['Severity']))
accidents_without_null['Severity'] = temp

def change_day(record):
    if record == 0:
        return str('monday')
    elif record == 1:
        return str('tuesday')
    if record == 2:
        return str('wednesday')
    elif record == 3:
        return str('thursday')
    if record == 4:
        return str('friday')
    elif record == 5:
        return str('saturday')
    if record == 6:
        return str('sunday')

temp = list(map(change_day,accidents_without_null['Start_Time']))
accidents_without_null['Start_Time'] = temp

for column in accidents_without_null.columns:
    if column in ['Start_Time']:
      dummies = pd.get_dummies(accidents_without_null[column],drop_first=False)
      accidents_without_null = pd.concat([accidents_without_null,dummies],axis=1)
      accidents_without_null.drop(column,axis=1,inplace=True)

In [736]:
accidents_without_null['Severity']

0          1
1          0
2          0
3          0
4          1
          ..
2845337    0
2845338    0
2845339    0
2845340    0
2845341    0
Name: Severity, Length: 606668, dtype: int64

In [732]:
accidents_without_null.to_csv('pre-processed_dataset_pre_dummies.csv')

In [733]:
for column in accidents_without_null.columns:
    if column in ['Weather_Condition']:
      dummies = pd.get_dummies(accidents_without_null[column],drop_first=False)
      accidents_without_null = pd.concat([accidents_without_null,dummies],axis=1)
      accidents_without_null.drop(column,axis=1,inplace=True)

In [734]:
accidents_without_null.shape

(606668, 125)

In [735]:
accidents_without_null.to_csv('pre-processed_dataset_no_state.csv')